In [6]:
from InputXGrad import InputXGradient
from gradcam import GradCAM
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
torch.set_default_tensor_type(torch.DoubleTensor)


In [2]:
x = torch.tensor([-1., 0., 1., 2., 3.], requires_grad=True)
#x.register_hook(lambda grad: print(grad))
z = torch.tensor([1., 2., 3., 4., 5.], requires_grad=True)
relu = nn.ReLU()
def func(x, z):
    return relu(1-x) + z
#x.requires_grad = True
y = func(x, z)

In [2]:
import numpy as np
import sys
sys.path.append('../')
from GNN import GNN
from featurization import MolTensorizer

In [7]:
smiles = 'COCc1cccc(-c2ccc(S(=O)(=O)Nc3cc(N4C[C@H](C)N[C@H](C)C4)ccc3OC)cc2)c1'
featurizer = MolTensorizer()
mol = featurizer.tensorize(smiles)
model = GNN(num_node_features=mol.x.shape[1],
            num_edge_features=mol.edge_attr.shape[1],)

In [8]:
explain_method = GradCAM(model, model.convs[-1])
att = explain_method.attribute((mol.x, mol.edge_attr), additional_forward_args=(mol.edge_index))

layer_evals (tensor([[0.5726, 0.0000, 0.9125,  ..., 0.0000, 0.0000, 1.3856],
        [0.5488, 0.0304, 0.0000,  ..., 0.3589, 0.0000, 1.2171],
        [0.5998, 0.0000, 0.3066,  ..., 0.0000, 0.2422, 1.4594],
        ...,
        [0.0000, 0.3468, 1.2298,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3446, 0.1967,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.5107, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<CatBackward0>),)
layer_evals[0] torch.Size([35, 128])


IndexError: tuple index out of range

In [63]:
#y.sum().backward()
optimizer.zero_grad()
y.sum().backward()
print(x.grad)


tensor([-1., -1., -0., -0., -0.])


In [17]:
explain_method = InputXGradient(func)
att = explain_method.attribute((x, z))

In [2]:
x = torch.tensor([1., 2., 3.], requires_grad=True).reshape(-1, 1)
edge_attr = torch.tensor([1., 1., 2., 2., 3., 3.], requires_grad=True).reshape(-1, 1)
edge_index = torch.tensor([[0, 1, 1, 2, 2, 0], [1, 0, 2, 1, 0, 2]], dtype=torch.long)
def func(x, edge_attr, edge_index):
    return torch.sum(x + scatter(edge_attr*x[edge_index[1]], edge_index[0])).reshape(-1, 1)
def scatter(x, index, dim=0):
    size = list(x.shape)
    size[dim] = index.max().item() + 1
    res = torch.zeros(size, dtype=x.dtype, device=x.device)
    return res.scatter_add_(dim, index[:, None].expand_as(x), x)
y = func(x, edge_attr, edge_index)

In [4]:
explain_method = InputXGradient(func)
att = explain_method.attribute((x, edge_attr), additional_forward_args=(edge_index,))

In [5]:
att

tensor([ 5.,  8., 18.], grad_fn=<MulBackward0>)

In [5]:
att

(tensor([ 5.,  8., 18.], grad_fn=<MulBackward0>),
 tensor([2., 1., 6., 4., 3., 9.], grad_fn=<MulBackward0>))